<a href="https://colab.research.google.com/github/Camouflage10/disease-spread-model/blob/Camouflage10-bagging/disease_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
import json

In [ ]:
#upload data
from matplotlib import text
x_original = pd.read_csv('dengue_features_train.csv')
y_original = pd.read_csv('dengue_labels_train.csv')
test =pd.read_csv('dengue_features_test.csv')

y_original=y_original.drop(columns=['city', 'year', 'weekofyear'])
x_original=x_original.drop(columns=['week_start_date','city', 'year'])

x_original['total_cases']=y_original['total_cases']
#x_original['year']=x_original['year'].astype(int)
#x_original['year'] = pd.Categorical(x_original.year)
#x_original['city'].replace(['sj', 'iq'],[0, 1], inplace=True)
#x_original['city'] = pd.Categorical(x_original.city)
x,y=x_original,y_original
test_removed=test[['city', 'year', 'week_start_date']]
test=test.drop(columns=['city', 'year', 'week_start_date'])
print(str(test.isna().sum()))
print('-----------------------')
print(str(x.isna().sum()))
original_length=len(x_original)
original_length

weekofyear                                0
ndvi_ne                                  43
ndvi_nw                                  11
ndvi_se                                   1
ndvi_sw                                   1
precipitation_amt_mm                      2
reanalysis_air_temp_k                     2
reanalysis_avg_temp_k                     2
reanalysis_dew_point_temp_k               2
reanalysis_max_air_temp_k                 2
reanalysis_min_air_temp_k                 2
reanalysis_precip_amt_kg_per_m2           2
reanalysis_relative_humidity_percent      2
reanalysis_sat_precip_amt_mm              2
reanalysis_specific_humidity_g_per_kg     2
reanalysis_tdtr_k                         2
station_avg_temp_c                       12
station_diur_temp_rng_c                  12
station_max_temp_c                        3
station_min_temp_c                        9
station_precip_mm                         5
dtype: int64
-----------------------
weekofyear                             

1456

In [ ]:
#tremove features with a corrilation of less than abs(mincorr)
minCorr=.2
corrMatrix = x.corr()
lastCol=corrMatrix['total_cases']
x_original=x_original.drop(columns=['total_cases'])
print(lastCol)
features=list(lastCol.loc[abs(lastCol) > minCorr].index)
features.remove('total_cases')
print(features)
x_fixed=x[features]
corrMatrix = x.corr()
for col in x_fixed.columns:
  print(col+":  "+str(x[col].isna().sum()))

weekofyear                               0.216452
ndvi_ne                                 -0.241376
ndvi_nw                                 -0.202235
ndvi_se                                 -0.168612
ndvi_sw                                 -0.196461
precipitation_amt_mm                    -0.038740
reanalysis_air_temp_k                    0.264952
reanalysis_avg_temp_k                    0.151637
reanalysis_dew_point_temp_k              0.142531
reanalysis_max_air_temp_k               -0.191345
reanalysis_min_air_temp_k                0.325252
reanalysis_precip_amt_kg_per_m2         -0.010031
reanalysis_relative_humidity_percent    -0.132452
reanalysis_sat_precip_amt_mm            -0.038740
reanalysis_specific_humidity_g_per_kg    0.129861
reanalysis_tdtr_k                       -0.278483
station_avg_temp_c                       0.116109
station_diur_temp_rng_c                 -0.237844
station_max_temp_c                      -0.039219
station_min_temp_c                       0.267109


In [ ]:
#make dictionary
columns=dict()
for col in x_fixed.columns:
  if x_fixed[col].isna().sum()>0:
    print(col+":  "+str(x_fixed[col].isna().sum()))
    columns[col]=x_fixed[col].isna().sum()
#order greatest to least
columns=dict(reversed(sorted(columns.items(), key=lambda item: item[1])))
columns


ndvi_ne:  194
ndvi_nw:  52
reanalysis_air_temp_k:  10
reanalysis_min_air_temp_k:  10
reanalysis_tdtr_k:  10
station_diur_temp_rng_c:  43
station_min_temp_c:  14


{'ndvi_ne': 194,
 'ndvi_nw': 52,
 'station_diur_temp_rng_c': 43,
 'station_min_temp_c': 14,
 'reanalysis_tdtr_k': 10,
 'reanalysis_min_air_temp_k': 10,
 'reanalysis_air_temp_k': 10}

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
grid_params={'n_estimators': [100,200],'max_features':['sqrt', 'log2', None]}
x=x_original
metrics=[]
models=dict()
#for loop thru the dict
for d in columns:
  print(d)
  x=x_original
  x=x[x[d].notna()]
  x=x.fillna(0)
  #temp x and y
  temp_y=x[d]
  #drop na for d and fill rest with 0s
  temp_x=x.drop(columns=d)
  #t and t split
  x_train, x_test, y_train, y_test = train_test_split(temp_x, temp_y, test_size=0.2)
  #svm
  forest=RandomForestRegressor()
  #gridsearch
  reg=GridSearchCV(estimator=forest,param_grid=grid_params)
  #fit best
  reg.fit(x_train,y_train)
  best=reg.best_estimator_
  print(reg.best_params_)
  #test model
  pred=best.predict(x_test)
  metrics.append(mean_squared_error(y_test,pred))
  #save model
  models[d]=best
metrics

ndvi_ne
{'max_features': None, 'n_estimators': 100}
ndvi_nw
{'max_features': 'sqrt', 'n_estimators': 100}
station_diur_temp_rng_c
{'max_features': None, 'n_estimators': 100}
station_min_temp_c
{'max_features': None, 'n_estimators': 200}
reanalysis_tdtr_k
{'max_features': 'log2', 'n_estimators': 200}
reanalysis_min_air_temp_k
{'max_features': 'log2', 'n_estimators': 100}
reanalysis_air_temp_k
{'max_features': None, 'n_estimators': 200}


[0.004825032225590244,
 0.004210726891853065,
 0.4774488994065933,
 0.4886861721453288,
 0.330166252182336,
 0.3485142655172503,
 0.02786654492400056]

In [ ]:
#predict and replace train
x=x_original
for d in columns: 
  x=x_original
  x_test=test
  print('-------------------------------------')
  print(d)
#take indexes of nan with
  x=x[x[d].isna()]
  x=x.drop(columns=d)

  x_test=x_test[x_test[d].isna()]
  x_test=x_test.drop(columns=d)
#remove rows with multipe nan or fill w avg or 0
  print("before: "+str(len(x_test)))
  x=x.fillna(0)
  x_test=x_test.fillna(0)
  if len(x)>0:
  #temp_x.apply(lambda x: x.fillna(x.mean()),axis=0)
#predict using the indexes of nan from x
    pred=models[d].predict(x)
#replace
    x[d]=pred
    x=x.replace(0,np.nan)
    x_original.update(x)
  if len(x_test)>0:
#predict using the indexes of nan from x
    pred=models[d].predict(x_test)
#replace
    x_test[d]=pred
    x_test=x_test.replace(0,np.nan)
    test.update(x_test)
    print("after: "+str(x_test[d].isna().sum()))
    print(len(x_test))
x_fixed_test=test[features]
fixed_len=len(x_fixed_test)
print(print("before: "+str(fixed_len)))
fixed_len=len(x_fixed_test.dropna())
print("after: "+str(fixed_len))
for col in x_fixed_test.columns:
  print(col+":  "+str(x_fixed_test[col].isna().sum()))

-------------------------------------
ndvi_ne
before: 43
after: 0
43
-------------------------------------
ndvi_nw
before: 11
after: 0
11
-------------------------------------
station_diur_temp_rng_c
before: 12
after: 0
12
-------------------------------------
station_min_temp_c
before: 9
after: 0
9
-------------------------------------
reanalysis_tdtr_k
before: 2
after: 0
2
-------------------------------------
reanalysis_min_air_temp_k
before: 2
after: 0
2
-------------------------------------
reanalysis_air_temp_k
before: 2
after: 0
2
before: 416
None
after: 416
weekofyear:  0
ndvi_ne:  0
ndvi_nw:  0
reanalysis_air_temp_k:  0
reanalysis_min_air_temp_k:  0
reanalysis_tdtr_k:  0
station_diur_temp_rng_c:  0
station_min_temp_c:  0


In [ ]:
x_original

,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,18.0,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,292.414286,299.8,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,19.0,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,300.9,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,20.0,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,295.434286,300.5,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,21.0,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,295.310000,301.4,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,22.0,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,295.821429,301.9,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451,21.0,0.342750,0.318900,0.256343,0.292514,55.30,299.334286,300.771429,296.825714,309.7,...,45.00,88.765714,55.30,18.485714,9.800000,28.633333,11.933333,35.4,22.4,27.0
1452,22.0,0.160157,0.160371,0.136043,0.225657,86.47,298.330000,299.392857,296.452857,308.5,...,207.10,91.600000,86.47,18.070000,7.471429,27.433333,10.500000,34.7,21.7,36.6
1453,23.0,0.247057,0.146057,0.250357,0.233714,58.94,296.598571,297.592857,295.501429,305.5,...,50.60,94.280000,58.94,17.008571,7.500000,24.400000,6.900000,32.2,19.2,7.4
1454,24.0,0.333914,0.245771,0.278886,0.325486,59.67,296.345714,297.521429,295.324286,306.1,...,62.33,94.660000,59.67,16.815714,7.871429,25.433333,8.733333,31.2,21.0,16.0


In [ ]:
#output new data
test[['city', 'year', 'week_start_date']]=test_removed
x_original.to_csv('train_x.csv')
test.to_csv('test.csv')


In [ ]:
#outputs a predicted model but there is no bagging model on this
#pred=model.predict(test)
#submition=test['city', 'year', 'weekofyear']
#submition['total_cases']=pred
#submition.to_csv('submition.csv')